## bpm_gibbs.py


The following is code for a Gibbs sampler for MCMC estimation of bipartite matching approach to record linkage.

## Bipartite matching approach to record linkage

Suppose that files $X_1$ and $X_2$ contain $n_1$ and $n_2$ records, respectively, and
without loss of generality that $n1 \geq n_2$. Denote also the number of entities represented in
both files as $n_{12}$, so that $n_2 \geq n_{12} \geq 0$.

The set of records coming from the two files can be represented as a *bipartite matching* and the parameter of interest is a matching matrix $\Delta$ of size $n_1\times n_2$ whose $(i,j)$th entry is defined as 

$$ \Delta_{ij} = \begin{cases} 1, & \text{ if records $i\in X_1$ and $j\in X_2$ refer to the same entity;} \\ 0, & \text{otherwise} \end{cases} $$

Sadinle (2017) uses a more compact representation called a *matching labeling*, which is useful when $n_1\times n_2$ is large.  Formally, the matching labeling is $Z = (Z_1, Z_2, \dots, Z_{n2})$, such that

$$ Z_j = \begin{cases} i, & \text{if records $i\in X_1$ and $j\in X_2$ refer to the same entity;} \\ n_1+j, & \text{if records $j\in X_2$ does not have a match in $X_1$ } \end{cases} $$

#### Advantages

#### Prior for $m$ and $u$ probabilities

It is expected that the probability of agreeing on an individual field of comparison is higher for matches than for nonmatches:

$$Pr(\gamma_{\ell}(a,b) = 1 |\ (a,b) \in M) > Pr(\gamma_{\ell}(a,b) = 1 |\ (a,b) \in U) $$ 

Use a dirichlet instead of independent betas. 

#### Beta Prior for Bipartite Matchings $Z$

This comes from Larsen (2005) and Sadinle (2017)/

Just as in the mixture model approach, the prior probability that $j \in X_2$ is:

$$ I(Z_j \leq n_1)  \overset{i.i.d}{\sim} \text{Bernoulli}(p_M)$$

where $p_M$ represents the proporiton of matches expected a priori as a fraction of the smallest file $X_2$.  Same as before, the hyperprior for $p_M$ is:

$$ p_M \sim \text{Beta}(\alpha_{M}, \beta_{M})$$ 

The prior on $p_M$ implies $n_{12}(Z) = \sum_{j=1}^{n_2} I(Z_j \leq n_1)$, the number of matches according to matching labeling $Z$ is distributed as:

$$n_{12}(Z) \sim \text{Beta-Binomial}(n_2, \alpha_{M}, \beta_{M}) $$ 

after marginalizing over $p_M$.

Conditioning on $\{I(Z_j \leq n_1)\}_{j=1}^{n_2}$, all possible bipartite matchings are taken to be equally likely, so $$Pr(Z\ |\ n_{12}) = \left(\frac{n_1!}{(n_1-n_{12})!}\right)^{-1}$$

These conditions imply the joint prior over $Z$:

$$Pr(Z\ |\ \alpha_M, \beta_M) = \frac{(n_1-n_{12}(Z))!}{n_1!}\frac{\text{Beta}(n_{12}(Z) + \alpha_M,\ n_2-n_{12}(Z) + \beta_M)}{\text{Beta}(\alpha_M, \beta_M)}$$





Finally 

$$\Gamma_{ij} | Z_j = i \overset{i.i.d}{\sim} M(m)$$

$$\Gamma_{ij}\ |\ Z_j \neq i \overset{i.i.d}{\sim} U(u) $$

$$ m_f \sim Dirichlet(\alpha_{\ell(0)}, \dots, \alpha_{\ell L}) $$

### Gibbs Sampler

Initialize match/nonmatch configuration $Z$. Tricks to do this.

1. Draw $p_M$ from $$ p_M\ |\ Z \sim \text{Beta}(\alpha_M + n_{M}(Z),\ \beta_M + n_{2} - n_{M}(Z)) $$ Note this is same as before. 

2. Draw $p_{M\ell}$ and $p_{U\ell}$ from their conditional distributions (same as before).

3. Use Metropolis-Hastings algorithm to draw values of $Z$ and $n_{12}(Z)$ from their full conditional distributions.  

The only difference is step 3! $I$ is no longer a bunch of Bernoullis, and assignment of $I(a,b)$ will now affect $I(a',b')$ which is desirable. 

### Python setup

In [1077]:
import numpy as np
import scipy

%matplotlib inline
from seaborn import plt
import pandas as pd 
import itertools

plt.rcParams['figure.figsize'] = (10, 5)

### 1. Updates for $p_M$


In [1170]:
def sample_pM(s, aM, bM):
    aNew = aM + s.nM
    bNew = bM + n2 - s.nM
    if ((aNew <= 0) or (bNew <=0)): print('neg. beta param')
    return np.random.beta(aNew,bNew)

### 2. Updates for $p_{M\ell},\ p_{U\ell}$

In [1171]:
def sample_pML(s, aML, bML):
    assert n2 == len(s.Z), 'Z got messed up'
    ones = np.array([1] * L)
    aSums = np.zeros(L)
    bSums = np.zeros(L)
    for x2 in s.matchedX2:
        matchInd = Gamma.index[(Gamma['i']==x2)&(Gamma['j']==s.Z[x2])].tolist()
        for y in matchInd:
            aSums += Gamma.loc[y]['gamma']
            bSums += (ones-Gamma.loc[y]['gamma'])
    aNew = aML + aSums
    bNew = bML + bSums
    if ((any(a <= 0 for a in aNew)) or (any(b <= 0 for b in bNew))): print('neg. beta param')
    return np.random.beta(aNew, bNew)

def sample_pUL(s, aUL, bUL):
    assert n2 == len(s.Z), 'Z got messed up'
    ones = np.array([1] * L)
    aSums = np.zeros(L)
    bSums = np.zeros(L)
    for x2 in range(n2):
        nonMatchInd = Gamma.index[(Gamma['i']==x2)&(Gamma['j']!=s.Z[x2])].tolist()
        for y in nonMatchInd:
            aSums += Gamma.loc[y]['gamma']
            bSums += (ones-Gamma.loc[y]['gamma'])
    aNew = aUL + aSums
    bNew = bUL + bSums
    if ((any(a <= 0 for a in aNew)) or (any(b <= 0 for b in bNew))): print('neg. beta param')
    return np.random.beta(aNew, bNew)

### 3. Updates for $Z, n_M$ 

These rely on many helper functions but below is a function that chooses the move type at random.

### Move #1: $\ n_m^* = n_m - 1$

Pick a record $j$ at random from the set of matched records $\{j: Z_j \leq n_{1}\}$ (with equal probability). 

In [1287]:
#gamma, n1, n2, pM, pML, pUL will be updated globally 
    
def move_1(s, pM, pML, pUL):
    
    assert len(s.matchedX2) == s.nM, 'nM is not calibrated correctly'
    assert len(s.matchedX2) == len(s.matchedX1), 'not bipartite matching'
    
    if len(s.matchedX2) == 0:
#         print('no matches to remove, try another move')
        return s
    
    old_state = make_state(s.Z, pM, pML, pUL) # save current Z
    
    #option 1 - randomly select i in matchedX2 and set to non-match 

    i = s.matchedX2[np.random.choice(s.nM)] # randomly select i in X2
    old_match = s.Z[i]                       # save its old match
    s.Z[i] = i + n1                          # set i's status to non-match in Z
    new_state = make_state(s.Z,pM, pML, pUL)              # make proposal state
    
    # calculate jump probability
    
    if s.nM == n2: # THIS THROWS ERROR divide by zero problem -- CHECK THIS
        const = 1
    else:
        const = s.nM/((n1-s.nM)*(n2-s.nM)) 
    
    pMH = min(1, new_state.llh*const/old_state.llh)
    
#     print('prob of jump is ' + str(pMH))
                                                  
    # choose jump or not
    accept = np.random.binomial(1, pMH)
    
    if accept == 1:  
        return new_state
        
    else:            
        return old_state

### Move #2: $\ n_m^* = n_m + 1$

Pick a record $j$ at random from the set of unmatched records $\{j: Z_j > n_{1}\}$ (with equal probability). 

NOT SURE IF CORRECT YET

In [1288]:
def move_2(s, pM, pML, pUL):
    
    assert len(s.matchedX2) == s.nM, 'nM is not calibrated correctly'
    assert len(s.matchedX2) == len(s.matchedX1), 'not bipartite matching'
    
    old_state = make_state(s.Z, pM, pML, pUL) # save current Z, fix any bugs
    
    if (len(old_state.unmatchedX2) == 0) or (len(old_state.unmatchedX1) == 0):
#         print('nothing left to match, try another move')
        return old_state
    
    # option 1 - randomly select which record pair to add

    addX2 = old_state.unmatchedX2[np.random.choice(len(old_state.unmatchedX2))] # randomly pick record to give match
    addX1 = old_state.unmatchedX1[np.random.choice(len(old_state.unmatchedX1))] # randomly pick its match
    
    s.Z[addX2] = addX1                                         # assign new match 
    new_state = make_state(s.Z, pM, pML, pUL)                                # make proposal state

    # calculate probability of jump
    
    if s.nM == 0:        # DIVIDE BY ZERO ERROR THINK ABOUT HOW TO FIX
        const = 1
        
    # is necessary? else if (s.nM == n1) or (n2 == s.nM):
    else:
        const = (n1-s.nM)*(n2-s.nM)/s.nM
        
    pMH = min(1, new_state.llh*const/old_state.llh)
#     print('prob of jump is ' + str(pMH))
    
    accept = np.random.binomial(1, pMH)
    
    if accept == 1:
         return new_state  
    else:
        return old_state

### Move #3: $\ n_M$ changed, but $Z$ altered

#### variation 1: Two matches switch pairings

In [1679]:
def move_3_v1(s, pM, pML, pUL):
    #updates state only 
    
    # in this type of move, the matched and unmatched don't change
    
    assert len(s.matchedX2) == s.nM, 'nM is not calibrated correctly'
    assert len(s.matchedX2) == len(s.matchedX1), 'not bipartite matching'
    
    old_state = make_state(s.Z, pM, pML, pUL) # save current Z, fix any bugs
    
    if old_state.nM < 2:
        # nothing to switch
        return old_state
    
    #Randomly select 2 matched pairs with prob 2/nm(nm-1)
    (i,k) = np.random.choice(old_state.nM, size=2, replace=False, p=None)
    j = old_state.Z[old_state.matchedX2[i]]
    l = old_state.Z[old_state.matchedX2[k]]
    
    # calculate jump probability pMH
    pMH = calc_pMH_move3(i,j,k,l, pML, pUL)
    
#     print('prob of jump is ' + str(pMH))
    accept = np.random.binomial(1, pMH)
    
    if accept == 1:
        
#         print('trying to pair (' + str(i) + ', ' + str(l) + ') and (' + str(k) + ',' + str(j) + ')')
        # flip entries in Z
        old_state.Z[old_state.matchedX2[i]] = l
        old_state.Z[old_state.matchedX2[k]] = j
        
        new_state = make_state(old_state.Z, pM, pML, pUL)
        return new_state
        
    else:
        return old_state

In [1680]:
def calc_pMH_move3(i, j, k, l, pML, pUL):
    gamma_il = Gamma[(Gamma['i']==int(i)) & (Gamma['j']==int(l))]['gamma'].values[0]
    gamma_kj = Gamma[(Gamma['i']==int(k)) & (Gamma['j']==int(j))]['gamma'].values[0]
    gamma_ij = Gamma[(Gamma['i']==i) & (Gamma['j']==j)]['gamma'].values[0]
    gamma_kl = Gamma[(Gamma['i']==k) & (Gamma['j']==l)]['gamma'].values[0]
    num = calc_pGammaM(gamma_il, pML)*calc_pGammaM(gamma_kj, pML) *\
            calc_pGammaU(gamma_ij, pUL)*calc_pGammaU(gamma_kl, pUL)
    denom = calc_pGammaM(gamma_ij, pML)*calc_pGammaM(gamma_kl, pML) *\
            calc_pGammaU(gamma_il, pUL)*calc_pGammaU(gamma_kj, pUL)
    return min(1, num/denom)


In [1705]:
Gamma = make_Gamma(n1,n2,pM,pML,pUL)
Z_init = make_Z_init(n1,n2)

trace, Z_trace = gibbs(Gamma, 500, init, hypers, Z_init)

performing type 1
performing type 2
performing type 3
performing type 1
performing type 3
performing type 1
performing type 1
performing type 3
performing type 1
performing type 2
performing type 2
performing type 1
performing type 1
performing type 2
performing type 2
performing type 1
performing type 2
performing type 1
performing type 2
performing type 3
performing type 2
performing type 3
performing type 1
performing type 3
performing type 1
performing type 3
performing type 2
performing type 1
performing type 2
performing type 1
performing type 3
performing type 3
performing type 2
performing type 3
performing type 3
performing type 1
performing type 1
performing type 3
performing type 2
performing type 2
performing type 1
performing type 2
performing type 3
performing type 1
performing type 3
performing type 2
performing type 1
performing type 3
performing type 1
performing type 2
performing type 2
performing type 1
performing type 3
performing type 2
performing type 2
performing

performing type 1
performing type 3
performing type 2
performing type 2
performing type 2
performing type 3
performing type 1
performing type 1
performing type 2
performing type 1
performing type 1
performing type 1
performing type 1
performing type 2
performing type 1
performing type 2
performing type 2
performing type 3
performing type 1
performing type 3
performing type 2
performing type 2
performing type 3
performing type 3
performing type 1
performing type 3
performing type 1
performing type 2
performing type 3
performing type 1
performing type 1
performing type 2
performing type 1
performing type 1
performing type 3
performing type 3
performing type 2
performing type 1
performing type 1
performing type 1
performing type 2


In [1704]:
trace_burnt = trace[100:499]
hist_plot = trace_burnt.hist(bins=30,layout=(2,4))

,pM,pML_1,pML_2,pUL_1,pUL_2
0,0.786988,0.311384,0.824594,0.432458,0.544377
1,0.724456,0.650894,0.496418,0.456725,0.495394
2,0.285147,0.148961,0.839024,0.443397,0.447560
3,0.438428,0.334413,0.232988,0.385424,0.521703
4,0.604739,0.582361,0.524104,0.393198,0.463352
5,0.469586,0.418353,0.801272,0.376702,0.389920
6,0.413602,0.793263,0.495200,0.440843,0.543315
7,0.102146,0.229169,0.369038,0.424794,0.414895
8,0.066952,0.681124,0.712320,0.343974,0.445892
9,0.097321,0.000150,0.443585,0.419923,0.581876


### Synthetic Data



making some data

### Maintaining $Z$ 

In [1658]:
class State(object):
    matchedX2 = []
    unmatchedX2 = []
    matchedX1 = []
    unmatchedX1 = []
    Z = []
    nM = 0
    llh = 1
    
    def __init__(self,Z):
        
        self.matchedX2 = [i for i in range(n2) if Z[i] < n1]
        self.matchedX1 = [i for i in Z if i < n1]
        self.nM = len(self.matchedX2)
        self.unmatchedX2 = np.delete([i for i in range(n2)], self.matchedX2)
        self.unmatchedX1 = [i for i in range(n1) if i not in Z]
        self.Z = Z

        if len(Z) != len(np.unique(Z)): 
            print_state(self)
            
        assert len(Z) == n2, 'invalid bpm'
        assert len(Z) == len(np.unique(Z)), 'invalid bpm'
               
def make_state(Z, pM, pML, pUL):
    state = State(Z)
    state.llh = calc_pNM_Z(state, pM, pML, pUL)
    return state

def print_state(s):
    print('Z: ' + str(s.Z))
    print('matchedX2: ' + str(s.matchedX2))
    print('matchedX1: ' + str(s.matchedX1))
    print('unmatchedX2: ' + str(s.unmatchedX2))
    print('unmatchedX1: ' + str(s.unmatchedX1))
    return

In [1659]:
#Functions for evaluating llh of Z

def calc_pGammaM(gammaInd,pML):
    assert len(gammaInd) == len(pML), 'dim do not match'
    return np.prod([(pML[l]**gammaInd[l])*(1-pML[l])**(1-gammaInd[l]) for l in range(len(pML))])

def calc_pGammaU(gammaInd,pUL):
    assert len(gammaInd) == len(pUL), 'dim do not match'
    return np.prod([(pUL[l]**gammaInd[l])*(1-pUL[l])**(1-gammaInd[l]) for l in range(len(pML))])

def calc_pGamma(s, pM, pML, pUL):
    # calculates P(gamma | Z, pML, pUL) for ENTIRE gamma (which has n1*n2 entries)
    pGamma = 1
    for index, row in Gamma.iterrows():
        if row['j'] == s.Z[row['i']]:
            pGamma = pGamma * calc_pGammaM(row['gamma'],pML)
        else:
            pGamma = pGamma*calc_pGammaU(row['gamma'],pUL)
            
    return pGamma

def calc_pNM_Z(s, pM, pML, pUL):
    #returns P(nM, Z | current params) ~ P(nM | pM) * P(Z | nM) * P(gamma | all param, Z) 
    pNM = binom.pmf(s.nM, n2, pM)  # p(nM | pM) ~ Binom(nM successes out of n2, w/ param pM)
    pZ = scipy.math.factorial(n1-s.nM)/scipy.math.factorial(n1)
    
    # up to here this is defined for nM = 0 
    pGamma = calc_pGamma(s, pM, pML, pUL) 
    
    return pNM * pZ * pGamma

### Writing the Gibbs sampler

In [1660]:
## specify initial values
init = {"pM": 0.5,
        "pML": [0.5]*L,
        "pUL": [0.5]*L}

## specify hyper parameters 
hypers = {"aM": 1,
          "bM": 1,
          "aML": [1]*L,
          "bML": [1]*L,
          "aUL": [1]*L,
          "bUL": [1]*L}

def check_valid_prior(init):
    assert (init["pM"] < 1) and (init['pM'] > 0), 'pM must be between 0,1'
    assert (0 not in init['pML']) and (1 not in init['pML']), 'elements of pML must be b/w 0,1'
    assert (0 not in init['pUL']) and (1 not in init['pUL']), 'elements of pML must be b/w 0,1'
    return 

In [1701]:
def gibbs(Gamma, iters, init, hypers, Z_init):
    # some assertions needed here
    assert len(init["pML"]) == len(init["pUL"]), 'dimensions of pML and pUL do not match'
    
    pM = init["pM"]
    pML = init["pML"]
    pUL = init["pUL"]
    nPar = 1 + len(pML) + len(pUL) 
    
    L = len(pML)
    numPair = Gamma.shape[0]
    
    trace = np.zeros((iters,nPar)) 
#     Z_trace = {}
    Z_trace = np.zeros((iters,n2)) 
    
    #initialize global state vars
    state = make_state(Z_init, pM, pML, pUL)
 
    for it in range(iters):
   
        # 1. Draw pM | Z ~ Beta(aM + nM(Z), bM + n2 - nM(Z)) 
        pM = sample_pM(state, hypers['aM'], hypers['bM']) 
        
        # 2. Draw pML | Z, pUL | Z
        gamma = Gamma['gamma']
        pML = sample_pML(state, hypers['aML'], hypers['bML'])
        pUL = sample_pUL(state, hypers['aUL'], hypers['bUL'])
        
#       3. Use Metropolis-Hastings algorithm to draw values of $Z$ and $n_{12}(Z)$ from
#         their full conditional distributions. 
        
        moveType = np.random.randint(3) + 1
        print('performing type ' + str(moveType))
        if moveType == 3:
            moveType = '3_v1'
        move = 'move_' + str(moveType) + '(state, pM, pML, pUL)'
        state = eval(move)
        
#         print('Z is ' + str(state.Z))
        
        trace[it,:] = np.append(np.append(pM, pML),pUL) # update trace
        Z_trace[it, :] = state.Z
        
#         for x in state.matchedX2:
#             if x not in Z_trace[x]:
#                 Z_trace[x] = {}
#                 Z_trace[x][Z[x]] = 0
#             Z_trace[x][Z[x]] += 1
            
    trace = pd.DataFrame(trace)
    Z_trace = pd.DataFrame(Z_trace)
    Z_trace = Z_trace.astype(int)
    
    pML_names = ['pML_' + str(i) for i in range(1,L+1)]
    pUL_names = ['pUL_' + str(i) for i in range(1,L+1)]
    trace.columns= ['pM'] + pML_names + pUL_names
    return trace, Z_trace

In [1688]:
pd.DataFrame(s.Z)

,0
0,10
1,8
2,12
3,13
4,14
5,0
6,7
7,17
8,1
9,4


### Likelihood functions

The functions below are used to evaluate the (non-normalized) posterior conditional distribution of $(n_{M}, Z)$ given the current parameter values according to:

$$ Pr(n_{M}, Z\ |\ \Gamma, \{p_{M\ell}, p_{U\ell}, \ell = 1,\dots,L \}, p_M, \alpha, \beta) \propto P(n_{M}\ |\ p_M) P(Z\ |\ n_{M}) P(\Gamma\ |\ Z, \{p_{M\ell}, p_{U\ell}, \ell = 1,\dots,L \}) $$

where $P(\Gamma\ |\ Z, etc.)$ = that annoying thing. 

In principle we could enumerate all possible $Z$ but that would be huge.  So instead we draw new values of $n_{12}^*, I^*$ incrementally. 



#### Function for making synthetic data

In [1662]:
from recordlinkage import datasets

def make_Gamma(n1, n2, pM, pML, pUL):
    nPair = n1 * n2
    L = len(pML)
    gamma=np.array(datasets.binary_vectors(nPair, int(pM*nPair), m=pML, u = pUL, random_state=113))
    i = [[i]*n1 for i in range(n2)]
    iVals = []
    for x in i:
        iVals += x
    jVals = [j for j in range(n1)] * n2
    Gamma = pd.DataFrame(
        {'gamma': [list(gamma[i]) for i in range(len(gamma))],
        'i': iVals, 
        'j': jVals})
    return Gamma

def make_Z_init(n1,n2):
    Z = [n1 + i for i in range(n2)]
    matchX1 = np.random.choice([i for i in range(n1)], size=int(.5*n2), replace=False, p=None)
    matchX2 = np.random.choice([i for i in range(n2)], size=int(.5*n2), replace=False, p=None)
    for i,x2 in enumerate(matchX2):
        Z[x2] = matchX1[i]
    return Z

In [1663]:
n1 = n2 = 10
pML = [.7, .7]
pUL = [.2, .2]
pM = .5
